The [ScienceBase Directory](https://www.sciencebase.gov/directory) is a system we set up a long time ago as part of the overall ScienceBase work to house a directory of people and organizations important in documenting data for items in ScienceBase. It was our own 'invention' in USGS, and we should arguably replace it at some point with a better tool. It is part of USGS Corporate Master Data and provides a central point of cleaning up some overall messiness in how we keep track of people who work for USGS over time and people from other organizations who we partner with in our work. It is also how we try to keep up with the even messier issue of changing organizational units in our own institution, in neighboring instititions within the Department of the Interior, and with other partner organizations. There are background synchronization processes that bring together and make some sense of basic personnel information from our corporate authentication mechanism as well as an internal business process management system that has various kinds of codes and identifiers used in these processes. There have been numerous attempts to make sense and best use of all of this internal information over time, and the pylinkedcmd is just one more bit of that work.

This notebook explores a set of functionality added to pylinkedcmd to help establish linkages from ScienceBase Directory person records to corresponding WikiData entities based on ORCID, which we can then exploit for further linked data coming from other sources developed through WikiData pathways. The motivation for this is an exercise in corporate capacity assessment we are just about to initiate where we wanted to build an algorithm to automatically assemble as much of an individual scientist's research record as possible to serve as context for survey responses. We know, anectodally, that no one accessible source of cataloged information has everything that any of us have contributed to. Official publication records, both journal articles and USGS reports, are mostly well cared for in the USGS Publications Warehouse, but they don't necessarily have everything for everyone and certainly not extending beyond USGS employment. ScienceBase and the USGS Science Data Catalog will have a lot of official data release items, but there will again be missing pieces. We haven't yet made solid connections to tap software control sources like GitHub, and our internal software control systems are less accessible to bot actions.

Exploring instances of human in WikiData with ORCIDs and indicating USGS employment was illuminating. There are lots of us with records in WikiData that have come together from a variety of databasing mechanisms that are constantly operating against WikiData. So, establishing a reasonably assured linkage to WikiData identities from ScienceBase Directory identities gives us a basis for exploring further exploitation of that resource pool. This notebook was used to run an initial baseline process and serves as an example for how some of pylinkedcmd can be used.

In [1]:
import pylinkedcmd
from IPython.display import display
import random
from datetime import datetime

from sqlite_utils import Database

# We put all the ScienceBase-specific stuff into its own class for clarity
lcmd_sb = pylinkedcmd.pylinkedcmd.Sciencebase()
lcmd_wikidata = pylinkedcmd.pylinkedcmd.Wikidata()

In [2]:
db = Database("orcid_to_wikidata.db")
tbl_all_orcids = db["all_orcids"]

person_package = list()
for index, row in enumerate(tbl_all_orcids.rows_where("date_updated IS NULL")):
#    if index > 100:
#        break
    person_package.append(row)
    
update_result = lcmd_sb.update_sb_person_identifiers(person_package)

update_records = [
    {
        "api": i["link"]["href"], 
        "date_updated": datetime.utcnow().isoformat()
    } for i in update_result["updatedPersonRecords"]
]
tbl_all_orcids.upsert_all(update_records, pk="api")
display(update_records)

User Name: sbristol@usgs.gov
········


[{'api': 'https://www.sciencebase.gov/directory/person/5500',
  'date_updated': '2020-08-06T17:44:56.202760'},
 {'api': 'https://www.sciencebase.gov/directory/person/11144',
  'date_updated': '2020-08-06T17:44:56.202776'},
 {'api': 'https://www.sciencebase.gov/directory/person/408',
  'date_updated': '2020-08-06T17:44:56.202781'},
 {'api': 'https://www.sciencebase.gov/directory/person/7088',
  'date_updated': '2020-08-06T17:44:56.202785'},
 {'api': 'https://www.sciencebase.gov/directory/person/10443',
  'date_updated': '2020-08-06T17:44:56.202789'},
 {'api': 'https://www.sciencebase.gov/directory/person/3011',
  'date_updated': '2020-08-06T17:44:56.202793'},
 {'api': 'https://www.sciencebase.gov/directory/person/23437',
  'date_updated': '2020-08-06T17:44:56.202797'},
 {'api': 'https://www.sciencebase.gov/directory/person/450',
  'date_updated': '2020-08-06T17:44:56.202801'},
 {'api': 'https://www.sciencebase.gov/directory/person/5159',
  'date_updated': '2020-08-06T17:44:56.202805'},


The ScienceBase Directory information model, for whatever reason, has both an identifiers object that can contain as many classified identifiers as someone includes in a record as well as a separate orcId property just for orcId identifiers. Part of what we do in this workflow is "fix" that issue, at least in terms of copying the ORCID values to the identifiers list. Here, we run a function we put in to pull together every ScienceBase Directory person record with an orcId value so that we can run through them and do other stuff. This takes care of query limits from the API to grab everything in one array.

In [ ]:
%%time
all_orcids = lcmd_sb.get_all_orcid()

In [ ]:
print(len(all_orcids))
display(all_orcids[random.randrange(len(all_orcids))])

We built our function to update the ScienceBase Directory person identifiers with both structured ORCID values and WikiData identifiers where available to be run from email address as the input variable as it seemed most reasonable as a starting point that could be used more broadly. The update function can run on a single email address but will also take a list. Here, we put together the entire list of emails we can run through and show a slice.

In [ ]:
all_emails = [i["email"] for i in all_orcids]
all_emails[:10]

The main function at work in this workflow will actually put everything together for a list of emails and execute the updates, prompting for ScienceBase credentials with permissions to update the directory (not very many users are authorized to do this). However, one part of that workflow will build the new person document with the lookup process to WikiData and is completely open and public for anyone to run. We show that here for a random person document, highlighting the new identifiers structure.

Actual linked open data practitioners will note that this is a pretty imperfect thing that we are setting up. We used an expedient of simply declaring, for our purposes, that we know what "ORCID" and "WikiData" type identifiers actually mean and understand how they can be used. We are mostly just using this for the purpose of establishing a connection to the "global knowledge graph" that we can then use to do other useful stuff.

In [ ]:
random_doc = lcmd_sb.get_identified_sb_person(all_emails[random.randrange(len(all_emails))])
display(random_doc[0])
display(random_doc[0]["identifiers"])

We can now run actual updates to add as many WikiData identifiers as we can reasonably establish to ScienceBase Person records so that we can exploit this linkage into the global knowledge graph (at least the WikiData version) further in downstream work. Note that this requires authentication to ScienceBase with sufficient permissions to modify the Directory.

In [ ]:
%%time
update_results = lcmd_sb.update_sb_person_identifiers(remaining_emails[:100])

In [ ]:
print("Updated Documents", len(update_results["updatedPersonRecords"]))
print("No Update Available", len(update_results["ignoredPersonRecords"]))
print("Number of WikiData links made", len([i for i in update_results["updatedPersonRecords"] if next((ident for ident in i["identifiers"] if ident["type"] == "WikiData"), None) is not None]))